## This Notebook generates two CSVs for Table 2
The first CSV is a table with mean infestation abundance for fish health zones each year as in Table 2, except for the 'Mean' row at the bottom.
The second CSV has the last two rows of the Table in the paper with counts of the months reported and number of farms in each zone.

In [31]:
from pathlib import Path
import pandas as pd

In [32]:
# define the data location constants
DATA_DIR = Path('..') / 'source_data'
FARM_INFO_PATH = DATA_DIR / 'industry_farm_details.csv'
FARM_SAMPLE_PATH = DATA_DIR / 'industry_farm_abundance.csv'

OUTPUT_DIR = Path('..') / 'output' / 'Table_2'

In [33]:
# import and merge the data
info_df = pd.read_csv(FARM_INFO_PATH)
sample_df = pd.read_csv(FARM_SAMPLE_PATH)

farm_df = pd.merge(info_df, sample_df, on='facility_id', how='left')

In [34]:
# restrict to wild outmigration months - March (3) to June (6)
farm_df = farm_df[farm_df.month.isin(range(3, 7))]

# rename column 'year' -> 'Year'
farm_df.rename(columns={'year': 'Year'}, inplace=True)

farm_df.head()

,facility_id,name,dfozone,latitude,longitude,company,region_name,Year,month,fish_selected,chalimus_ab,lep_motile_ab,lep_af_ab,cal_motile_ab,weight
0,1,Ahlstrom,3_1,49.778527,-124.154337,Grieg,Jervis and Sechelt Inlet,2009,5,120,0.0000,0.0167,0.0083,0.0000,1.0000
1,1,Ahlstrom,3_1,49.778527,-124.154337,Grieg,Jervis and Sechelt Inlet,2009,6,40,0.0000,0.0000,0.0000,0.0000,0.4793
6,1,Ahlstrom,3_1,49.778527,-124.154337,Grieg,Jervis and Sechelt Inlet,2011,3,120,0.2250,0.4000,0.2583,6.0167,0.2097
7,1,Ahlstrom,3_1,49.778527,-124.154337,Grieg,Jervis and Sechelt Inlet,2011,4,120,0.1917,0.4000,0.1167,1.2167,0.2337
8,1,Ahlstrom,3_1,49.778527,-124.154337,Grieg,Jervis and Sechelt Inlet,2011,5,120,0.0000,0.0833,0.0167,0.0750,0.2184


## Calculate the weighted mean motile abundance each year in each zone

In [35]:
# calculate the weighted lep_motile_ab for each farm in each year-month
farm_df['weighted_lep_motile_ab'] = farm_df.lep_motile_ab * farm_df.weight

# sum weighted abundance and weights over each year
farm_weight_group = farm_df.groupby(['dfozone', 'Year'])
farm_year_weight_df = farm_weight_group[['weighted_lep_motile_ab', 'weight']].sum().reset_index()

# weighted abundance / weight sum
farm_year_weight_df['mean_weighted_lep_motile_ab'] = \
    farm_year_weight_df.weighted_lep_motile_ab / farm_year_weight_df.weight

farm_year_weight_df.head()

,dfozone,Year,weighted_lep_motile_ab,weight,mean_weighted_lep_motile_ab
0,2_3,2005,4.098165,4.0002,1.024490
1,2_3,2006,8.559096,4.0001,2.139720
2,2_3,2007,4.217373,4.0000,1.054343
3,2_3,2008,4.171165,4.0000,1.042791
4,2_3,2009,2.316495,4.0000,0.579124


In [36]:
year_zone_abundance_df = pd.pivot_table(farm_year_weight_df, values='mean_weighted_lep_motile_ab',
                                        index='Year', columns='dfozone', aggfunc='first').reset_index()

year_zone_abundance_df.head()

dfozone,Year,2_3,2_4,3_1,3_2,3_3,3_4,3_5
0,2004,NaN,NaN,NaN,NaN,4.533794,NaN,NaN
1,2005,1.024490,0.481279,0.110915,1.579893,1.367901,0.717995,0.233987
2,2006,2.139720,0.597255,0.121569,1.458852,0.900993,0.876854,0.157242
3,2007,1.054343,0.497918,NaN,0.525048,0.457153,0.969240,0.199342
4,2008,1.042791,0.402636,NaN,1.718428,0.269718,0.904075,0.235625


In [37]:
# rename the zones to use '.' instead of '_'
zone_rename_dict = {name: name.replace('_', '.') for name in year_zone_abundance_df.columns}

year_zone_abundance_df.rename(columns=zone_rename_dict, inplace=True)
year_zone_abundance_df.head()

dfozone,Year,2.3,2.4,3.1,3.2,3.3,3.4,3.5
0,2004,NaN,NaN,NaN,NaN,4.533794,NaN,NaN
1,2005,1.024490,0.481279,0.110915,1.579893,1.367901,0.717995,0.233987
2,2006,2.139720,0.597255,0.121569,1.458852,0.900993,0.876854,0.157242
3,2007,1.054343,0.497918,NaN,0.525048,0.457153,0.969240,0.199342
4,2008,1.042791,0.402636,NaN,1.718428,0.269718,0.904075,0.235625


In [38]:
# export the table to a CSV
year_zone_abundance_df.to_csv(OUTPUT_DIR / 'Table_2_abundance.csv', index=False)

## Calculate the number of reported months and number of farms in each zone

In [39]:
num_farm_zone_group = farm_df.groupby('dfozone')
num_farms_in_zone_df = num_farm_zone_group['facility_id'].agg('nunique').reset_index()

num_farms_in_zone_df.head()

,dfozone,facility_id
0,2_3,14
1,2_4,15
2,3_1,7
3,3_2,19
4,3_3,23


In [40]:
num_farms_in_zone_df['label'] = 'Farms (N)'
num_farms_in_zone_df.rename(columns={'facility_id': 'count'}, inplace=True)
num_farms_in_zone_df.head()

,dfozone,count,label
0,2_3,14,Farms (N)
1,2_4,15,Farms (N)
2,3_1,7,Farms (N)
3,3_2,19,Farms (N)
4,3_3,23,Farms (N)


In [41]:
# only select months where the number of fish sampled is more than 0
num_months_zone_group = farm_df[farm_df.fish_selected > 0].groupby('dfozone')
num_months_in_zone_df = num_months_zone_group['month'].agg('count').reset_index()

num_months_in_zone_df.head()

,dfozone,month
0,2_3,609
1,2_4,604
2,3_1,224
3,3_2,605
4,3_3,979


In [42]:
num_months_in_zone_df['label'] = 'Months (N)'
num_months_in_zone_df.rename(columns={'month': 'count'}, inplace=True)
num_months_in_zone_df.head()

,dfozone,count,label
0,2_3,609,Months (N)
1,2_4,604,Months (N)
2,3_1,224,Months (N)
3,3_2,605,Months (N)
4,3_3,979,Months (N)


In [43]:
num_months_farms_df = pd.concat([num_farms_in_zone_df, num_months_in_zone_df])

In [44]:
num_per_zone_df = pd.pivot_table(num_months_farms_df, values='count', index='label', columns=['dfozone'],
                                 aggfunc='first', fill_value=0).reset_index()

num_per_zone_df.head()

dfozone,label,2_3,2_4,3_1,3_2,3_3,3_4,3_5
0,Farms (N),14,15,7,19,23,10,8
1,Months (N),609,604,224,605,979,347,259


In [45]:
# export the table to a CSV file
num_per_zone_df.to_csv(OUTPUT_DIR / 'Table_2_months_farms.csv')